In [4]:
import json
import re

In [5]:
import sys
sys.path.insert(0, "../textToLatex")
from pytexit import py2tex

In [6]:
supported_operators = ["**", "/", "*", "+", ">", "<", "="]
supported_word_operators = ["sqrt", "abs", "inf"]
replace = {"⋅" : "*", "−" : "-", "^" : "**", "𝑥" : "x", "𝑎" : "a", "𝑏" : "b", "𝑦" : "y", "–": "-", "≥" : ">=", "≤": "<=", "∪" : "U"}
conditionally_replace = {"[" : "(", "]" : ")"}
regex = re.compile("|".join(map(re.escape, replace.keys())))

#Figure out way to deal with equal signs
def preprocess_text_to_latex(text, tutoring=False, stepMC= False):
    text = str(text)
    text = regex.sub(lambda match: replace[match.group(0)], text)
    if not re.findall("[\[|\(][-\d\s\w]+,[-\d\s\w]+[\)|\]]", text): #Checking to see if there are coordinates/intervals before replacing () with []
        text = regex.sub(lambda match: conditionally_replace[match.group(0)], text)
    
    
    #Account for space in sqrt(x, y)
    text = re.sub(r"sqrt[\s]?\(([^,]+),[\s]+([^\)])\)", r"sqrt(\g<1>,\g<2>)", text)
    text = re.sub(r"sqrt(?:\s*)?\(", r"sqrt(", text)
    text = re.sub(r"abs(?:\s*)?\(", r"abs(", text)
    text = re.sub("\([\s]*([-\d]+)[\s]*,[\s]*([-\d]+)[\s]*\)", "(\g<1>,\g<2>)", text) #To account for coordinates
    for operator in supported_operators:
        text = re.sub("(\s?){0}(\s?)".format(re.escape(operator)), "{0}".format(operator), text)
        
    words = text.split()
    latex = False
    for i in list(range(len(words))):
        word = words[i]
        
        if any([op in word for op in supported_operators]) or any([op in word for op in supported_word_operators]):
            punctuation = re.findall("[\?\.,:]+}$", word) #Capture all the punctuation at the end of the sentence
            if punctuation:
                punctuation = punctuation[0]
            else:
                punctuation = ""
            word = re.sub("[\?\.,:]+}$", "", word)
            try:                
                sides = re.split('(=|U|<=|>=)', word)
                sides = [handle_word(side) for side in sides]
                new_word = ""
                if tutoring and stepMC:
                    new_word = "$$" + "".join(sides) + "$$"
                    #sides = ["$$" + side + "$$" for side in sides] 
                elif tutoring:
                    new_word = "$$" + "".join([side.replace("\\", "\\\\") for side in sides]) + "$$"
                    #sides = ["$$" + side.replace("\\", "\\\\") + "$$" for side in sides]
                else:
                    new_word = "<InlineMath math=\"" + "".join(sides) + "\"/>"
                    #sides = ["<InlineMath math=\"" + side + "\"/>" for side in sides]
                #new_word = "=".join(sides)
                new_word += punctuation
                latex=True
                words[i] = new_word
                
            except Exception as e:
                print("This failed")
                print(word)
                print(e)
                pass
    text = " ".join(words)
    return text, latex

def handle_word(word):
    latex_dic = {"=": "=", "U": " \cup ", "<=" : " \leq ", ">=" : " \geq "}
    if word in latex_dic:
        return latex_dic[word]
    
    coordinates = re.findall("[\(|\[][-\d\s]+,[-\d\s]+[\)|\]]",word)
    if coordinates:
        word = re.sub("inf", r"\\infty", word)
        return word
    
    word = re.sub("\+/-", "pm(a)", word)
    
    original_word = word
    scientific_notation = re.findall("\(?([\d]{2,})\)?\*([\d]{2,})\*\*", word)
    word = re.sub(":sqrt", ": sqrt", word)
    square_roots = re.findall(r"sqrt\(([^,]*)\,([^\)]*)\)", word)
    word = re.sub(",", "", word)
    for root in square_roots:
        word = re.sub(r"sqrt\("+root[0]+root[1]+"\)", r"sqrt("+root[0]+","+root[1]+")", word)
    #word = re.sub(r"sqrt\(([^,]*)\,([^\)]*)\)", r"sqrt(\g<1>:\g<2>)", "sqrt(2, 3)")
    word = re.sub(r"([\w])(\(+[\w])", "\g<1>*\g<2>", word)
    word = re.sub(r"(\)+)([\w])", "\g<1>*\g<2>", word)
    word = re.sub(r"(\))(\()", "\g<1>*\g<2>", word)
    word = re.sub(r"([0-9]+)([a-zA-Z])", "\g<1>*\g<2>", word)
    #word = re.sub( r"([a-zA-Z])(?=[a-zA-Z])" , r"\1*" , word)
    word = re.sub(r"sqrt\*", r"sqrt", word)
    word = re.sub(r"abs\*", r"abs", word)
    word = re.sub(r"pm\*", r"pm", word)
    word = py2tex(word, simplify_output=False)
    
    #Here do the substitutions for the things that py2tex can't handle
    for item in scientific_notation:
        word = re.sub(item[0] + "\{" + item[1] + "\}", item[0] + "\\\\times {" + item[1] + "}", word)
    word = re.sub(r"\\operatorname{pm}\\left\(a\\right\)(\\times)?", r"\\pm ", word)
    
    return word[2:-2]

In [9]:
preprocess_text_to_latex("(z*sqrt(2))**5/(z*sqrt(2))")

<IPython.core.display.Latex object>

$$\frac{{\left(z \sqrt{2}\right)}^5}{z \sqrt{2}}$$


('<InlineMath math="\\frac{{\\left(z \\sqrt{2}\\right)}^5}{z \\sqrt{2}}"/>',
 True)

In [7]:
scaff_dic = {"mc": "string", "numeric": "TextBox", "algebra": "TextBox", "string": "string"}

def handle_answer_type(answer_type):
    if answer_type == "mc":
        return "string", "MultipleChoice"
    elif answer_type == "string":
        return answer_type, "TextBox"
    elif answer_type == "algebra" or answer_type == "numeric":
        return "arithmetic", "TextBox"
    else:
        print(answer_type)
        raise Exception('answer type not correct' + answer_type)


def create_scaffold(step, hint_id, title, body, answer_type, answer, mc_answers, dependencies=0.0, images="", subhints=[], hint_dic={}):
    title, title_latex = preprocess_text_to_latex(title, True)
    body, body_latex = preprocess_text_to_latex(body, True)
    
    scaffold_id = step + "-" + hint_id
    for image in images:
        body += "\\n##{0}##".format(image)

    if type(dependencies) == str:
        try:
            dependencies = json.dumps([hint_dic[hint_id] for hint_id in dependencies.split(",")])
        except Exception as e:
            print("Key error")
            print(step, hint_id, title, body)
            raise Exception("hint key error")
    else:
        dependencies = "[]"
    
    answer_type, problem_type = handle_answer_type(answer_type)
    if answer_type == "arithmetic":
        answer = re.sub("\*\*", "^", answer)
    scaff_ans = "[\"" + str(answer) + "\"]"

    
    if type(mc_answers) != float:
        mc_answers = json.dumps([preprocess_text_to_latex(mc_answer, True)[0] for mc_answer in mc_answers.split("|") if mc_answer])
        answer = preprocess_text_to_latex(answer, True)[0]
    
    

    scaff_obj = "id: \"{0}\", type: \"scaffold\", problemType: \"{1}\", answerType: \"{2}\", hintAnswer: {3}, dependencies: {4}".format(scaffold_id, problem_type, answer_type, scaff_ans, dependencies)
    scaff_obj += ", title: \"{0}\"".format(title)
    scaff_obj += ", text: \"{0}\"".format(body)
    
    
    if type(mc_answers) == str:
        scaff_obj += ", choices: {0}".format(mc_answers)
    if len(subhints) > 0:
        subhint_text = ""
        for subhint in subhints:
            subhint_text += subhint + ", "
        subhint_text = subhint_text[:-2]
        subhint_text = "[" + subhint_text + "]"    
        scaff_obj += ", subHints: {0}".format(subhint_text)

    scaff_obj = "{" + scaff_obj + "}"
    return scaff_obj, scaffold_id

In [8]:
print(create_scaffold("slope2a", "h1", "Slope Example (Part 1)", "what is the answer to x**2+5", "mc", "3", mc_answers="3**2|4**2|5**2|6**2", images=["figure1.gif"]))

<IPython.core.display.Latex object>

$$x^2+\left(5\right)$$


<IPython.core.display.Latex object>

$${\left(3\right)}^2$$


<IPython.core.display.Latex object>

$${\left(4\right)}^2$$


<IPython.core.display.Latex object>

$${\left(5\right)}^2$$


<IPython.core.display.Latex object>

$${\left(6\right)}^2$$
('{id: "slope2a-h1", type: "scaffold", problemType: "MultipleChoice", answerType: "string", hintAnswer: ["3"], dependencies: [], title: "Slope Example (Part 1)", text: "what is the answer to $$x^2+\\\\left(5\\\\right)$$\\n##figure1.gif##", choices: ["$${\\\\\\\\left(3\\\\\\\\right)}^2$$", "$${\\\\\\\\left(4\\\\\\\\right)}^2$$", "$${\\\\\\\\left(5\\\\\\\\right)}^2$$", "$${\\\\\\\\left(6\\\\\\\\right)}^2$$"]}', 'slope2a-h1')
